In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 31.8 MB/s 
     |████████████████████████████████| 163 kB 56.4 MB/s 
     |████████████████████████████████| 7.6 MB 60.4 MB/s 


In [25]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

acceptability_tokenizer = AutoTokenizer.from_pretrained('RussianNLP/ruRoBERTa-large-rucola')
acceptability_model = AutoModelForSequenceClassification.from_pretrained('RussianNLP/ruRoBERTa-large-rucola')

In [26]:
if torch.cuda.is_available():
    acceptability_model.cuda()

In [29]:
def predict_acceptability(text):
    inputs = acceptability_tokenizer(text, max_length=128, add_special_tokens=False, return_tensors='pt').to(acceptability_model.device)
    with torch.inference_mode():
        logits = acceptability_model(**inputs).logits
        probas = torch.nn.Softmax(dim=1)(logits)[0].cpu().detach().numpy()
    return probas[0]

In [4]:
import pandas as pd
import ast

df = pd.read_csv('train.tsv', sep='\t', names=['text', 'context', 'y'])

In [30]:
predict_acceptability(df['text'][0])

0.51507103

In [5]:
from tqdm import tqdm
import numpy as np

tqdm.pandas()

df['acceptability'] = df['text'].progress_apply(predict_acceptability)

  1%|          | 282/28901 [00:03<05:39, 84.29it/s]


KeyboardInterrupt: 

In [13]:
from scipy.stats import spearmanr

print(spearmanr(df['acceptability'], df['y']))

SpearmanrResult(correlation=-0.06612011466392806, pvalue=2.2526719459690325e-29)


In [21]:
df[df['acceptability'] == 1]

,text,context,y,acceptability,new_acceptability
2,марусь включи пожалуйста елена вальяк гипноз д...,"[{""user"": ""маруся через десять минут будильник...",0,1,1
3,да о том что я тебя очень уважаю люблю,"[{""user"": """", ""marusia"": ""Если вам нужна помощ...",0,1,1
5,включи песню король и шут кукла колдуна,"[{""user"": ""включи песню король и шут песня кол...",0,1,1
7,а можешь ты управлять устройством дебилка блин...,"[{""user"": ""маруся а можешь управлять человеком...",0,1,1
8,это например муз вот уж не за что,"[{""user"": ""какие планы на ближайшие дни"", ""mar...",1,1,1
...,...,...,...,...,...
28893,двенадцать часов ночи я тебя скачала,"[{""user"": ""ты знаешь артура михалияна он вчера...",0,1,1
28896,так где мой нож,"[{""user"": """", ""marusia"": """"}, {""user"": ""ты уме...",1,1,1
28898,маруся я очень тебя люблю,"[{""user"": ""маруся очень очень я тебя люблю"", ""...",0,1,1
28899,голову вас зафиксировала камера,"[{""user"": """", ""marusia"": """"}, {""user"": ""ну дав...",1,1,1
